# LightGBM Hyperparameter Sweep 20211031
For optimizing lv2 predictions off of a lv1 with 5 random seeds

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = True

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random
import multiprocessing
import pickle

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"sweep_lightgbm_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
#     datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = root/'models'
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'optuna_studies'
    altdatapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/')
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath, altdatapath]:
        pth.mkdir(exist_ok=True)

In [5]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # !pip install --upgrade xgboost

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    # !pip install category_encoders
    # !pip install catboost
    !pip install --upgrade -q lightgbm

    # lighgbm gpu compatible
    # !git clone --recursive https://github.com/Microsoft/LightGBM
    # ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
    
    # # this part is from https://github.com/rapidsai/gputreeshap/issues/24
    # !pip install cmake --upgrade
    # # !pip install sklearn --upgrade
    # !git clone --recursive https://github.com/dmlc/xgboost
    # %cd /content/xgboost
    # !mkdir build
    # %cd build
    # !cmake .. -DUSE_CUDA=ON
    # !make -j4
    # %cd /content/xgboost/python-package
    # !python setup.py install --use-cuda --use-nccl
    # !/opt/bin/nvidia-smi
    # !pip install shap
    

Now, non-stdlib imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from optuna.integration.wandb import WeightsAndBiasesCallback
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
# from sklearn.impute import KNNImputer, StandardImputer
# import timm

import seaborn as sns

# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


from optuna.samplers import TPESampler
import optuna
# import catboost
from sklearn.utils import resample
import sklearn.metrics

from lightgbm.basic import LightGBMError

Now, datapath setup

In [7]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [8]:

# n_trials = int(1000)
SEED = 42

In [9]:
# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
seed_everything(SEED)

## Ex-Model Config

In [10]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'lightgbm',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
#     'optuna_trials': 100,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for Level 2 LightGBM on GPU with Optuna using original dataset with KMeans 8-clustering and synthetic features.",
    'config': exmodel_config,
}

## Data Setup

**TODO** Write some conditional logic here to automate it -- possibly as part of a sklearn.*pipeline

In [11]:
# train_source = datapath/'train.feather'
# df = pd.read_feather(path=datapath/'train.feather')
# y = np.array(df.target)
# dump(y, filename=datapath/'y.joblib')
# del df

# y = load(datapath/'y.joblib')
y = load(predpath/'5folds_rs42_oof_y.joblib')

# df.index.name = 'id'
# y_train = df.target
# features = [x for x in df.columns if x != 'target']
# X_train = df[features]
# # X.index.name = 'id'
# # y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# del df, X_train, y_train

# load the Boruta-filtered green-zone 98 features (based on 200 iterations of the algo)
# train_source = '/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/X_boruta_200iter_filtered_green.joblib'
# X = load(train_source)

# train_source = altdatapath/'train-WITH-KMeans_8cluster_ninit50_maxiter1000_rs42-AND-synthetic.feather' #'X_boruta_shap_200trials.feather'
# train_source = predpath/'stacking_manual_20211020_104938_X_orig+KMeans8+synth_oof_lv1.feather'
train_source = altdatapath/'oof_lv1_5rs.feather'
exmodel_config['train_source'] = str(train_source)
X = pd.read_feather(path=train_source)
if 'target' in X.columns:
    X = X.drop(['target'], axis=1)

# exmodel_config['feature_count'] = len(X.columns)
exmodel_config['feature_count'] = X.shape[1]
exmodel_config['instance_count'] = X.shape[0]

# exmodel_config['feature_generator'] = None
# exmodel_config['feature_generator'] = "Summary statistics"

# exmodel_config['train_source'] = str(train_source)
# test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
# X_test = pd.read_feather(path=test_source)
# X_test = X_test.iloc[:, 1:]
# X_test = np.array(X_test)

In [12]:
# train_source = datapath/'train.feather'
# df = pd.read_feather(path=train_source)
# df.index.name = 'id'
# y_train = df.target
# features = [x for x in df.columns if x != 'target']
# X_train = df[features]
# # X.index.name = 'id'
# # y.index.name = 'id'
# X = np.array(X_train)
# y = np.array(y_train)

# # del df, X_train, y_train


# # exmodel_config['feature_count'] = len(X.columns)
# exmodel_config['feature_count'] = X.shape[1]
# exmodel_config['instance_count'] = X.shape[0]

# # exmodel_config['feature_generator'] = None
# # exmodel_config['feature_generator'] = "Summary statistics"

# exmodel_config['train_source'] = str(train_source)

In [13]:
# test_source = datapath/'test.feather'
# exmodel_config['test_source'] = str(test_source)
# X_test = pd.read_feather(path=test_source)
# # X_test = X_test.iloc[:, 1:]

In [14]:
# X_test = np.array(X_test)

# Experiment setup

In [15]:
# wandb_kwargs = {
#     # wandb config:
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'project': '202109_Kaggle_tabular_playground',
#     'tags': ['sweep'],
#     'notes': "Sweep for CatBoost using Optuna",
#     'config': exmodel_config,
# }

In [16]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
       
    # else:
    #     print("No best study yet")
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=int(SEED), shuffle=True)
    # create wrappers for the training and validation partitions
    # train_pool = catboost.Pool(X_train, y_train)
    # valid_pool = catboost.Pool(X_valid, y_valid)
    
    # experimental parameters
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 900, 7000),
        'max_depth' : trial.suggest_int('depth', 3, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.4),               
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 30),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 30),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1), # aka bagging_fraction
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss']),
        'min_child_samples': trial.suggest_int('min_child_samples', 4, 75),
        'num_leaves': trial.suggest_int('num_leaves', 50, 250),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1), # aka feature_fraction
    }  

    # instantiate the model, with some parameters locked in, and experimnental ones passed via splat 
#     try:
#         model = LGBMClassifier(
#             objective='binary',
#             random_state=SEED,
#     #                     device_type='cpu',
#     #                     n_jobs=-1,
#     #                 eval_metric='auc',
#             device_type='gpu',
#             max_bin=63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
#     #                 gpu_use_dp=False, # forces use of single precision rather than double for better perf, esp on consumer Nvidia chips
#             **params)

#     except LightGBMError:
#         print("GPU not working")
    model = LGBMClassifier(
        objective='binary',
        random_state=SEED,
        device_type='cpu',
        n_jobs=-1,
#                 eval_metric='auc',
#                 device_type='gpu',
#                 max_bin=63, # 15 might be even better for GPU perf, but depends on dataset -- see https://lightgbm.readthedocs.io/en/latest/GPU-Performance.html
#                 gpu_use_dp=False, # forces use of single precision rather than double for better perf, esp on consumer Nvidia chips
        **params)

    model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()])
    # generate predictions
    preds = model.predict_proba(X_valid)[:,1]
    # rounds to the nearest integer, and the nearest even in case of _.5s

    # Evaluation
    valid_auc = roc_auc_score(y_valid, preds)
    print('ROC AUC Score =', valid_auc)
    wandb.log({'valid_auc': valid_auc,
              })

    return valid_auc

In [17]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

<ipython-input-17-69ea9289a2cf>:1: ExperimentalWarning: WeightsAndBiasesCallback is experimental (supported from v2.9.0). The interface can change in the future.
  wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find sweep_lightgbm_20211031.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [18]:
study = optuna.create_study(direction = "maximize", 
                            sampler = TPESampler(seed=int(SEED)), 
                            study_name=f"lightgbm_lv2-X_orig_{datetime.now().strftime('%Y%m%d')}")

# study = load('/home/sf/code/kaggle/tabular_playgrounds/oct2021/optuna_studies/optuna_lightgbm_study_65trials_20211005.joblib')


[I 2021-10-31 16:20:16,635] A new study created in memory with name: lightgbm_lv2-X_orig_20211031


In [19]:
import lightgbm

In [20]:

for x in range(1,250):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc], catch=(lightgbm.basic.LightGBMError,)) #n_jobs = multiprocessing.cpu_count())
#     print(f"{x+26} trials complete")
    dump(study, filename=datapath/f"optuna_lightgbm_study_{x}trials_{datetime.now().strftime('%Y%m%d')}.joblib")
#     dump(study.best_trial.params, filename=datapath/f'optuna_lightgbm_study_best-thru-{x*5}trials_20210927.joblib')

[I 2021-10-31 16:36:31,532] Trial 0 finished with value: 0.8554744943328172 and parameters: {'n_estimators': 3185, 'depth': 10, 'learning_rate': 0.0802956743641955, 'reg_alpha': 0.4789240251631179, 'reg_lambda': 0.004994757081068292, 'subsample': 0.5779972601681014, 'boosting_type': 'dart', 'min_child_samples': 54, 'num_leaves': 54, 'colsample_bytree': 0.9849549260809971}. Best is trial 0 with value: 0.8554744943328172.
wandb: WARNING Step must only increase in log calls.  Step 0 < 3186; dropping {'n_estimators': 3185, 'depth': 10, 'learning_rate': 0.0802956743641955, 'reg_alpha': 0.4789240251631179, 'reg_lambda': 0.004994757081068292, 'subsample': 0.5779972601681014, 'boosting_type': 'dart', 'min_child_samples': 54, 'num_leaves': 54, 'colsample_bytree': 0.9849549260809971, 'value': 0.8554744943328172}.


ROC AUC Score = 0.8554744943328172


[I 2021-10-31 16:38:18,449] Trial 1 finished with value: 0.8568742389365649 and parameters: {'n_estimators': 5978, 'depth': 4, 'learning_rate': 0.002972483637079397, 'reg_alpha': 0.0066240595682091315, 'reg_lambda': 0.023021277110080198, 'subsample': 0.762378215816119, 'boosting_type': 'goss', 'min_child_samples': 14, 'num_leaves': 108, 'colsample_bytree': 0.6831809216468459}. Best is trial 1 with value: 0.8568742389365649.
wandb: WARNING Step must only increase in log calls.  Step 1 < 9165; dropping {'n_estimators': 5978, 'depth': 4, 'learning_rate': 0.002972483637079397, 'reg_alpha': 0.0066240595682091315, 'reg_lambda': 0.023021277110080198, 'subsample': 0.762378215816119, 'boosting_type': 'goss', 'min_child_samples': 14, 'num_leaves': 108, 'colsample_bytree': 0.6831809216468459, 'value': 0.8568742389365649}.


ROC AUC Score = 0.8568742389365649


[I 2021-10-31 16:41:06,222] Trial 2 finished with value: 0.8567423885688474 and parameters: {'n_estimators': 3682, 'depth': 9, 'learning_rate': 0.003307982168695265, 'reg_alpha': 0.20058106556780586, 'reg_lambda': 0.4490677404109781, 'subsample': 0.5232252063599989, 'boosting_type': 'gbdt', 'min_child_samples': 72, 'num_leaves': 244, 'colsample_bytree': 0.9041986740582306}. Best is trial 1 with value: 0.8568742389365649.
wandb: WARNING Step must only increase in log calls.  Step 2 < 12848; dropping {'n_estimators': 3682, 'depth': 9, 'learning_rate': 0.003307982168695265, 'reg_alpha': 0.20058106556780586, 'reg_lambda': 0.4490677404109781, 'subsample': 0.5232252063599989, 'boosting_type': 'gbdt', 'min_child_samples': 72, 'num_leaves': 244, 'colsample_bytree': 0.9041986740582306, 'value': 0.8567423885688474}.


ROC AUC Score = 0.8567423885688474


[I 2021-10-31 16:49:00,189] Trial 3 finished with value: 0.8568121134133031 and parameters: {'n_estimators': 2758, 'depth': 3, 'learning_rate': 0.06031361827702156, 'reg_alpha': 0.09345791438428568, 'reg_lambda': 0.0035186816415472676, 'subsample': 0.7475884550556351, 'boosting_type': 'dart', 'min_child_samples': 51, 'num_leaves': 112, 'colsample_bytree': 0.7600340105889054}. Best is trial 1 with value: 0.8568742389365649.
wandb: WARNING Step must only increase in log calls.  Step 3 < 15607; dropping {'n_estimators': 2758, 'depth': 3, 'learning_rate': 0.06031361827702156, 'reg_alpha': 0.09345791438428568, 'reg_lambda': 0.0035186816415472676, 'subsample': 0.7475884550556351, 'boosting_type': 'dart', 'min_child_samples': 51, 'num_leaves': 112, 'colsample_bytree': 0.7600340105889054, 'value': 0.8568121134133031}.


ROC AUC Score = 0.8568121134133031


KeyboardInterrupt: 

In [ ]:
# dump(study, filename=datapath/f"optuna_lightgbm_26trials-complete_{datetime.now().strftime('%Y%m%d')}.joblib")
# dump(study.best_trial.params, filename=datapath/f"optuna_lightgbm_all-500trials-best_{datetime.now().strftime('%Y%m%d')}.joblib")
# pickle.dump(study.best_trial.params, open('CatBoost_Hyperparameter.pickle', 'wb'))
# print('CatBoost Hyperparameter:', study.best_trial.params)

In [21]:
study.best_trial.params

{'n_estimators': 5978,
 'depth': 4,
 'learning_rate': 0.002972483637079397,
 'reg_alpha': 0.0066240595682091315,
 'reg_lambda': 0.023021277110080198,
 'subsample': 0.762378215816119,
 'boosting_type': 'goss',
 'min_child_samples': 14,
 'num_leaves': 108,
 'colsample_bytree': 0.6831809216468459}

In [22]:
wandb.log({'lightgbm_params': study.best_trial.params})
wandb.finish()

valid_auc,▁█▇█
valid_auc,0.85681


In [ ]:
optuna.visualization.plot_parallel_coordinate(study)